In [1]:
import pandas as pd
from ib_insync import *
from collections import namedtuple
from ibapi.client import EClient
from ibapi.wrapper import EWrapper

In [2]:
Trade = namedtuple("Trade",["date","symbol","price","quantity"])

In [3]:
class SignalGenerator:
    def __init__(self,signals):
        self.signals = signals
        
    def generate_signals(self):
        for signal in self.signals:
            yield signal

In [4]:
def preprocess_data(func):
    def wrapper(self, df):
        #leaving it for future use
        return func(self, df)

    return wrapper

In [5]:
class IBClient(EWrapper,EClient):
    def __init__(self,addr,port,client_id):
        EClient.__init__(self,self)
        self.connect(addr,port,client_id)
        self.reqId = 0
        self.data = {}
        self.portfolio = {'Cash':200000,'Position':{}}
        self.strategy_params = {'buy_treshold':0.02,'sell_treshold':-0.02}
        self.trade_contract = None
        
    def error(self,reqId, errorCode, errorString):
        print(f"Error {errorCode}:{errorString}")
        
    @preprocess_data
    def historicalDataEnd(self, reqId, start, end, df):
        signals = self.generate_signals(df)
        signal_generator = SignalGenerator(signals)
        self.execute_trades(signal_generator,df)
        
    def generate_signals(self,df):
        df['SMA_50'] = df['Close'].rolling(50).mean()
        df['SMA_200'] = df['Close'].rolling(200).mean()
        
        df['Signal'] = 0
        buy_treshold = self.strategy_params['buy_treshold']
        sell_treshold = self.strategy_params['sell_treshold']
        
        df.loc[(df['Close']>df['SMA_50'] * (1+buy_treshold)) & (df['Close'] > df['SMA_200']), 'Signal'] = 1
        df.loc[(df['Close']<df['SMA_50'] * (1+sell_treshold))& (df['Close']<df['SMA_200']),'Signal'] = -1
        
        return df['Signal']
    
    def execute_traders(self, signal_generator,df):
        trade = []
        for signal in signal_generator.generate_signals():
            date = signal[0]
            trade = None
            if signal[1]==1:
                trade = Trade(date,"BUY",df.loc[date,'Close'],1)
            elif signal[1]==-1:
                trade = Trade(date,"SELL",df.loc[date,'Close'],-1)
                
            if trade is not None:
                trades.append(trade)
        self.update_portfolio(trades)
        
    def update_portfolio(self,trades):
        for trade in trades:
            if trade.quantity>0:
                if self.portfolio['Cash'] >=trade.price*trade.quantity:
                    self.portfolio['Cash'] -= trade.price * trade.quantity
                    self.portfolio['Positions'].setdefault(trade.date, []).append(trade)
                    print(f"{trade.symbol} {trade.quantity} shares bought at {trade.price}")
                    
            
            elif trade.quantity < 0:
                date = pd.Timestamp.now().date()
                if date in self.portfolio['Positions']:
                    positions = self.portfolio['Positions'][date]
                    position_quantity = sum([p.quantity for p in positions])
                    if position_quantity >= abs(trade.quantity):
                        self.portfolio['Cash'] += trade.price * abs(trade.quantity)
                        remaining_quantity = abs(trade.quantity)
                        updated_positions = []
                        for position in positions:
                            if position.quantity >= remaining_quantity:
                                position.quantity -= remaining_quantity
                                updated_positions.append(position)
                                remaining_quantity = 0
                                break
                            else:
                                remaining_quantity -= position.quantity
                        self.portfolio['Positions'][date] = updated_positions
                        if len(self.portfolio['Positions'][date]) == 0:
                            del self.portfolio['Positions'][date]
                            
    def start_trading(self, contract, duration="1 D", barsize="1 min"):
        self.trade_contract = contract
        bars = self.reqHistoricalData(
            self.reqId,
            contract,
            endDateTime="",
            durationStr=duration,
            barSizeSetting=barsize,
            whatToShow="TRADES",
            useRTH=1,
            formatDate=1,
            keepUpToDate=True,
            chartOptions=[],
        )
        self.reqId += 1



ib_host = "127.0.0.1"
ib_port = 7497
ib_client_id = 1

contract = Stock("AAPL", "SMART", "USD")


ib_client = IBClient(ib_host, ib_port, ib_client_id)
ib_client.start_trading(contract)


util.startLoop()